# World Info Tuning
mkultra provides a specialized trainer for generating soft World Info*.

This trainer takes datapoints in a "call and response" format. The "call" is a fixed prompt (e.g. "A detailed description of the Spider Tank:") that should elicit the "response" (a few paragraphs of information about the Spider Tank). Multiple responses can be specified for each call, but loss is only calculated against the responses.

The trainer will optimize a soft prompt such that the call is followed by the response. The combined call and response is moved randomly within the context window to prevent overfitting, and the gap is filled with random tokens.

This setup may work for other tasks, but if your needs are significantly different then it is recommended to see tuning_finetune.ipynb for tips on rolling your own.

*Short prompt infixes for AI text adventures and writing.
Describes a character, subject, artistic direction, etc. with the intent of steering the output to consistently make use of the details in question.

In [1]:
#@title Setup for Colab only
#!pip install transformers
#!pip install git+git://github.com/corolla-johnson/mkultra.git#egg=mkultra

In [2]:
from transformers.pipelines import pipeline
from mkultra.models.tuning import GPT2PromptTuningLM
from mkultra.tokenizers import GPT2TokenizerFast
from mkultra.soft_prompt import SoftPrompt
from transformers import Adafactor
import torch

In [3]:
# Use an mkultra prompt tuning LM and a standard tokenizer.
model = GPT2PromptTuningLM.from_pretrained("gpt2").to("cuda")
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

In [4]:
# Set the length of the soft prompt.
# The paper does not recomment going over 100 tokens.
model.initialize_soft_prompt(n_tokens=20)

call_str = "Detailed description of Emma Violence:\n"

response_str = ("Emma Violence is a British cybernetic assassin who is known for her "
                "elegant style and high-profile targets. She is described to be a perfectionist "
                "in her work, often going above and beyond the call of duty. "
                "Despite her cold, ruthless, and calculating nature, she has a warm and "
                "motherly side that she only shows to a select few people. She has two guns "
                "implanted to her forearms and can utilize them both with deadly accuracy.")

call = tokenizer(call_str, return_tensors="pt").input_ids.cuda()
response = tokenizer(response_str, return_tensors="pt").input_ids.cuda()

input_ids = torch.cat( [call, response], dim=1 )

ignore_call = torch.full((1,call.shape[-1]),-100).cuda()

labels = torch.cat( [ ignore_call, response ], dim=1 )

print(input_ids.shape)
print(labels.shape)

# Multiple answers can be written for a single question.
# answers = [[]]

# The components are assembled like this:
# |soft prompt|random tokens|question|answer


torch.Size([1, 97])
torch.Size([1, 97])


In [5]:
model.train()
optimizer = Adafactor(params=model.get_soft_params())

In [14]:
#@title Training
#@markdown 4000+ for "gpt2"
#@markdown
#@markdown 200+ for "GPT-Neo-2.7B"

iterations = 1000#@param{type:"number"}

for i in range(iterations):
  optimizer.zero_grad()
  output = model(input_ids=input_ids, labels=labels)
  loss = output.loss
  loss.backward()
  optimizer.step()
  if i%5 == 0:
    print(f"{i}: Loss: {loss}")

0: Loss: 0.6359873414039612
5: Loss: 0.6187944412231445
10: Loss: 0.7337299585342407
15: Loss: 0.5554460287094116
20: Loss: 0.7459617853164673
25: Loss: 0.6622592210769653
30: Loss: 0.6346929669380188
35: Loss: 0.5845086574554443
40: Loss: 0.4881477355957031
45: Loss: 0.736186146736145
50: Loss: 0.6162692904472351
55: Loss: 0.5107298493385315
60: Loss: 0.5365223288536072
65: Loss: 0.6170713901519775
70: Loss: 0.679900050163269
75: Loss: 0.5693861246109009
80: Loss: 0.49865373969078064
85: Loss: 0.3966021239757538
90: Loss: 0.5641685724258423
95: Loss: 0.544960081577301
100: Loss: 1.4968494176864624
105: Loss: 0.607020914554596
110: Loss: 0.4740757942199707
115: Loss: 0.5228590965270996
120: Loss: 0.5585851669311523
125: Loss: 0.604408323764801
130: Loss: 0.4761580526828766
135: Loss: 0.6136701703071594
140: Loss: 0.4362785220146179
145: Loss: 0.5728248357772827
150: Loss: 0.4585609436035156
155: Loss: 0.5957559943199158
160: Loss: 0.4781897962093353
165: Loss: 0.6148995161056519
170: L

In [25]:
basic_output = model.generate(
    call,
    do_sample=True,
    min_length=call.shape[-1] + 100,
    max_length=call.shape[-1] + 100,
    temperature=0.1,
    top_p = 0.9,
    repetition_penalty = 1.0,
    pad_token_id=tokenizer.eos_token_id
)
print(tokenizer.decode(basic_output[0]))

Detailed description of Emma Violence:
Emma Violence is a British cybernetic assassin who is known for her elegant and high-profile targets. She is portrayed to be a perfectionist in her work, often taking down targets in a manner that is almost certain to mock execution. Despite her cold temperate personality, she is a dedicated a strict and dedicated killer. She has two guns implanted to her forearms and moles, and nce, and is able to utilize them both with deadly precision and accuracy. She is known for her


In [7]:
# WorldInfoTrainer will help prevent overfitting by moving the target
# up and down the context, filling the rest with random tokens.
# Also, only the 'target' string is used to calculate loss.
# Note that very low loss may not be acceptable for the description task.
"""
trainer = WorldInfoTrainer(
    model=model
    optimizer=Adafactor(params=model.get_soft_params())
    blocks = [block],
    max_spacing=0, # Setting this to 0 for illustrative purposes
    min_loss=0.4
)

trainer.train()
"""

'\ntrainer = WorldInfoTrainer(\n    model=model\n    optimizer=Adafactor(params=model.get_soft_params())\n    blocks = [block],\n    max_spacing=0, # Setting this to 0 for illustrative purposes\n    min_loss=0.4\n)\n\ntrainer.train()\n'

## Best Practices for World Info
Unlike the finetuning example, this is fresh territory.
Here are potential techniques to explore:
- Define bad_words_ids (e.g. square brackets) and surround the response in them to keep the output from repeating the training data verbatim.
- Rather than describing the subject in grammatically correct sentences, try a word cloud approach.
- Rather than describing the subject, provide implicit examples of to write about it ("John doffed his black top hat and cleaned it with his white handkerchief")
- Consider the use of the subject's proper noun vs its pronouns. (Should you start every sentence with "John Doe", or a mix of "John" and "He"?)
- Definitely use the min_loss parameter to arrest the tuning.